# <div style="text-align:center; border-radius:30px 30px; padding:13px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#1c4f9e; overflow:hidden"><b>Car Price Prediction with Linear Regression</b></div>

In this project, we aim to use linear regression to predict car prices based on a dataset provided by Kaggle. The analysis explores the relationship between various car features and their corresponding prices to develop an accurate predictive model.

In [2]:
#load Packages
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


In [3]:
df = pd.read_csv("Dataset/car_detail.csv")

In [4]:
df.head()

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,Seller Type,Engine,Max Power,Max Torque,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Pune,Grey,First,Corporate,1198 cc,87 bhp @ 6000 rpm,109 Nm @ 4500 rpm,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,Ludhiana,White,Second,Individual,1248 cc,74 bhp @ 4000 rpm,190 Nm @ 2000 rpm,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Lucknow,Maroon,First,Individual,1197 cc,79 bhp @ 6000 rpm,112.7619 Nm @ 4000 rpm,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Mangalore,Red,First,Individual,1197 cc,82 bhp @ 6000 rpm,113 Nm @ 4200 rpm,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Mumbai,Grey,First,Individual,2393 cc,148 bhp @ 3400 rpm,343 Nm @ 1400 rpm,RWD,4735.0,1830.0,1795.0,7.0,55.0


In [6]:
df.describe()

,Price,Year,Kilometer,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
count,2.059000e+03,2059.000000,2.059000e+03,1995.000000,1995.000000,1995.000000,1995.000000,1946.000000
mean,1.702992e+06,2016.425449,5.422471e+04,4280.860652,1767.991980,1591.735338,5.306266,52.002210
std,2.419881e+06,3.363564,5.736172e+04,442.458507,135.265825,136.073956,0.822170,15.110198
min,4.900000e+04,1988.000000,0.000000e+00,3099.000000,1475.000000,1165.000000,2.000000,15.000000
25%,4.849990e+05,2014.000000,2.900000e+04,3985.000000,1695.000000,1485.000000,5.000000,41.250000
50%,8.250000e+05,2017.000000,5.000000e+04,4370.000000,1770.000000,1545.000000,5.000000,50.000000
75%,1.925000e+06,2019.000000,7.200000e+04,4629.000000,1831.500000,1675.000000,5.000000,60.000000
max,3.500000e+07,2022.000000,2.000000e+06,5569.000000,2220.000000,1995.000000,8.000000,105.000000


In [7]:
df.isna().sum()

Make                    0
Model                   0
Price                   0
Year                    0
Kilometer               0
Fuel Type               0
Transmission            0
Location                0
Color                   0
Owner                   0
Seller Type             0
Engine                 80
Max Power              80
Max Torque             80
Drivetrain            136
Length                 64
Width                  64
Height                 64
Seating Capacity       64
Fuel Tank Capacity    113
dtype: int64

In [8]:
df.shape

(2059, 20)

In [13]:
#Delete row with NaN
df_clean = df.dropna(subset = ['Engine','Max Power', 'Max Torque', 'Drivetrain', 'Length', 'Width', 'Height', 'Seating Capacity', 'Fuel Tank Capacity'])

In [14]:
df_clean.isna().sum()

Make                  0
Model                 0
Price                 0
Year                  0
Kilometer             0
Fuel Type             0
Transmission          0
Location              0
Color                 0
Owner                 0
Seller Type           0
Engine                0
Max Power             0
Max Torque            0
Drivetrain            0
Length                0
Width                 0
Height                0
Seating Capacity      0
Fuel Tank Capacity    0
dtype: int64

In [16]:
df['Model'].unique().sum()

'Amaze 1.2 VX i-VTECSwift DZire VDIi10 Magna 1.2 Kappa2Glanza GInnova 2.4 VX 7 STR [2016-2020]Ciaz ZXiCLA 200 Petrol SportX1 xDrive20d M SportOctavia 1.8 TSI Style Plus AT [2017]Terrano XL (D)Elite i20 Sportz 1.2Kwid 1.0 RXT [2016-2019]Ciaz Alpha Hybrid 1.5 AT [2018-2020]Harrier XZ [2019-2020]Polo GT TSICelerio ZXi AMT [2019-2020]Alto 800 LXi (O)Baleno Alpha AutomaticWagon R ZXi 1.2 AMTCreta 1.6 E PetrolS-Presso VXi AMTVento Comfortline Petrol ATSantro Sportz AMT [2018-2020]Venue SX 1.0 (O) Petrol [2019-2020]Alto LXi CNGRitz Zxi BS-IVCreta 1.6 SX Plus PetrolBrio V MTElite i20 Asta 1.2WR-V VX MT PetrolVenue SX 1.0 Turbo iMTCreta 1.6 SX Plus ATEcosport Titanium+ 1.0L EcoBoostCR-V 2.4 ATSantro GL (CNG)Tiago Revotron XE [2016-2019]Tigor Revotorq XTInnova 2.4 G 8 STR [2016-2017]GLC 220 d ProgressiveCreta SX 1.6 CRDi (O)M-Class ML 250 CDICorolla Altis VL AT PetrolA4 2.0 TDI (177bhp) Premium PlusSLK-Class SLK 200 KThar CRDe 4x4 ACBaleno Delta 1.2Celerio VXi AMTCiaz VDi SHVSCiaz VXiCreta SX Pl

In [18]:
df['Location'].unique()

array(['Pune', 'Ludhiana', 'Lucknow', 'Mangalore', 'Mumbai', 'Coimbatore',
       'Bangalore', 'Delhi', 'Raipur', 'Kanpur', 'Patna', 'Vadodara',
       'Hyderabad', 'Yamunanagar', 'Gurgaon', 'Jaipur', 'Deoghar', 'Agra',
       'Goa', 'Warangal', 'Jalandhar', 'Noida', 'Ahmedabad', 'Mohali',
       'Navi Mumbai', 'Ghaziabad', 'Kolkata', 'Zirakpur', 'Nagpur',
       'Thane', 'Faridabad', 'Ranchi', 'Chandigarh', 'Amritsar',
       'Chennai', 'Udupi', 'Panvel', 'Jamshedpur', 'Aurangabad',
       'Rudrapur', 'Nashik', 'Varanasi', 'Salem', 'Dehradun', 'Valsad',
       'Haldwani', 'Dharwad', 'Surat', 'Indore', 'Karnal', 'Panchkula',
       'Mysore', 'Rohtak', 'Ambala Cantt', 'Samastipur', 'Unnao',
       'Purnea', 'Bhubaneswar', 'Kheda', 'Kollam', 'Meerut', 'Ernakulam',
       'Kharar', 'Mirzapur', 'Bhopal', 'Gorakhpur', 'Guwahati',
       'Allahabad', 'Muzaffurpur', 'Faizabad', 'Kota', 'Pimpri-Chinchwad',
       'Dak. Kannada', 'Ranga Reddy', 'Bulandshahar', 'Roorkee',
       'Siliguri'], dty

In [19]:
df['Max Torque'].unique()

array(['109 Nm @ 4500 rpm', '190 Nm @ 2000 rpm', '112.7619 Nm @ 4000 rpm',
       '113 Nm @ 4200 rpm', '343 Nm @ 1400 rpm', '130 Nm @ 4000 rpm',
       '300 Nm @ 1200 rpm', '400 Nm @ 1750 rpm', '250 Nm @ 1250 rpm',
       '200 Nm @ 1900 rpm', '115 Nm @ 4000 rpm', '91 Nm @ 4250 rpm',
       '138 Nm @ 4400 rpm', '350 Nm @ 1750 rpm', '175 Nm @ 1500 rpm',
       '90 Nm @ 3500 rpm', '69 Nm @ 3500 rpm', '154 Nm @ 4850 rpm',
       '153 Nm @ 3800 rpm', '99 Nm @ 4500 rpm', '172 Nm @ 1500 rpm',
       '78 Nm @ 3500 rpm', '113 Nm @ 4500 rpm', '110 Nm @ 4800 rpm',
       '265 Nm @ 1900 rpm', '170 Nm @ 1400 rpm', nan, '96 Nm @ 3000 rpm',
       '114 Nm @ 3500 rpm', '140 Nm @ 1800 rpm', '400 Nm @ 1400 rpm',
       '260 Nm @ 1500 rpm', '500 Nm @ 1600 rpm', '173 Nm @ 4000 rpm',
       '380 Nm @ 1750 rpm', '240@3000', '247 Nm @ 1800 rpm',
       '200 Nm @ 1750 rpm', '200 Nm @ 1200 rpm', '290 Nm @ 1800 rpm',
       '270 Nm @ 1350 rpm', '145 Nm @ 4600 rpm', '180 Nm @ 2000 rpm',
       '101 Nm @ 3000 rpm

In [20]:
df_drop = df.drop(['Model', 'Location','Engine', 'Max Power', 'Max Torque'], axis=1)

So I decided to delete all these columns because there are too many possibilities, and it is overwhelming for the model. The goal is to simplify the dataset to help the model focus on the most relevant features and improve its performance. 

In [21]:
df_drop

,Make,Price,Year,Kilometer,Fuel Type,Transmission,Color,Owner,Seller Type,Drivetrain,Length,Width,Height,Seating Capacity,Fuel Tank Capacity
0,Honda,505000,2017,87150,Petrol,Manual,Grey,First,Corporate,FWD,3990.0,1680.0,1505.0,5.0,35.0
1,Maruti Suzuki,450000,2014,75000,Diesel,Manual,White,Second,Individual,FWD,3995.0,1695.0,1555.0,5.0,42.0
2,Hyundai,220000,2011,67000,Petrol,Manual,Maroon,First,Individual,FWD,3585.0,1595.0,1550.0,5.0,35.0
3,Toyota,799000,2019,37500,Petrol,Manual,Red,First,Individual,FWD,3995.0,1745.0,1510.0,5.0,37.0
4,Toyota,1950000,2018,69000,Diesel,Manual,Grey,First,Individual,RWD,4735.0,1830.0,1795.0,7.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,Mahindra,850000,2016,90300,Diesel,Manual,White,First,Individual,FWD,4585.0,1890.0,1785.0,7.0,70.0
2055,Hyundai,275000,2014,83000,Petrol,Manual,White,Second,Individual,FWD,3495.0,1550.0,1500.0,5.0,32.0
2056,Ford,240000,2013,73000,Petrol,Manual,Silver,First,Individual,FWD,3795.0,1680.0,1427.0,5.0,45.0
2057,BMW,4290000,2018,60474,Diesel,Automatic,White,First,Individual,RWD,4936.0,1868.0,1479.0,5.0,65.0


## Feature encoding 

In [23]:
df_drop['Make'].unique()

array(['Honda', 'Maruti Suzuki', 'Hyundai', 'Toyota', 'Mercedes-Benz',
       'BMW', 'Skoda', 'Nissan', 'Renault', 'Tata', 'Volkswagen', 'Ford',
       'Audi', 'Mahindra', 'MG', 'Jeep', 'Porsche', 'Kia', 'Land Rover',
       'Volvo', 'Maserati', 'Jaguar', 'Isuzu', 'Fiat', 'MINI', 'Ferrari',
       'Mitsubishi', 'Datsun', 'Lamborghini', 'Chevrolet', 'Ssangyong',
       'Rolls-Royce', 'Lexus'], dtype=object)

In [25]:
df_encoded = pd.get_dummies(df_drop, 
                          columns=['Make', 'Fuel Type', 'Transmission','Color','Owner','Seller Type','Drivetrain'],
                          prefix=['mrq', 'Fuel', 'Transmission','Color','Owner','Seller Type','Drivetrain'])

In [26]:
df_encoded.shape

(2059, 81)

In [27]:
df_encoded.head()

,Price,Year,Kilometer,Length,Width,Height,Seating Capacity,Fuel Tank Capacity,mrq_Audi,mrq_BMW,...,Owner_Fourth,Owner_Second,Owner_Third,Owner_UnRegistered Car,Seller Type_Commercial Registration,Seller Type_Corporate,Seller Type_Individual,Drivetrain_AWD,Drivetrain_FWD,Drivetrain_RWD
0,505000,2017,87150,3990.0,1680.0,1505.0,5.0,35.0,False,False,...,False,False,False,False,False,True,False,False,True,False
1,450000,2014,75000,3995.0,1695.0,1555.0,5.0,42.0,False,False,...,False,True,False,False,False,False,True,False,True,False
2,220000,2011,67000,3585.0,1595.0,1550.0,5.0,35.0,False,False,...,False,False,False,False,False,False,True,False,True,False
3,799000,2019,37500,3995.0,1745.0,1510.0,5.0,37.0,False,False,...,False,False,False,False,False,False,True,False,True,False
4,1950000,2018,69000,4735.0,1830.0,1795.0,7.0,55.0,False,False,...,False,False,False,False,False,False,True,False,False,True


In [29]:
bool_columns = df_encoded.select_dtypes(include=['bool']).columns
df_encoded[bool_columns] = df_encoded[bool_columns].astype(int)

In [30]:
df_encoded.head()

,Price,Year,Kilometer,Length,Width,Height,Seating Capacity,Fuel Tank Capacity,mrq_Audi,mrq_BMW,...,Owner_Fourth,Owner_Second,Owner_Third,Owner_UnRegistered Car,Seller Type_Commercial Registration,Seller Type_Corporate,Seller Type_Individual,Drivetrain_AWD,Drivetrain_FWD,Drivetrain_RWD
0,505000,2017,87150,3990.0,1680.0,1505.0,5.0,35.0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,450000,2014,75000,3995.0,1695.0,1555.0,5.0,42.0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,220000,2011,67000,3585.0,1595.0,1550.0,5.0,35.0,0,0,...,0,0,0,0,0,0,1,0,1,0
3,799000,2019,37500,3995.0,1745.0,1510.0,5.0,37.0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,1950000,2018,69000,4735.0,1830.0,1795.0,7.0,55.0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [35]:
df_encoded['Year'] = pd.to_datetime(df_encoded['Year'])

In [36]:
df_encoded['Year'].dtypes

dtype('<M8[ns]')